In [ ]:
#Thanks to: https://www.curiousily.com/posts/object-detection-on-custom-dataset-with-tensorflow-2-and-keras-using-python/

In [ ]:
#MLFlow tutorial https://databricks.com/blog/2018/09/21/how-to-use-mlflow-to-reproduce-results-and-retrain-saved-keras-ml-models.html

In [1]:
from mlflow import log_metric, log_param, log_artifact

In [1]:
!pip3 install sklearn

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
    100% |████████████████████████████████| 19.4MB 13kB/s eta 0:00:01   37% |████████████                    | 7.3MB 2.2MB/s eta 0:00:06    54% |█████████████████▍              | 10.6MB 3.3MB/s eta 0:00:03    95% |██████████████████████████████▍ | 18.4MB 3.4MB/s eta 0:00:01
    100% |████████████████████████████████| 296kB 410kB/s ta 0:00:01


In [3]:
import pandas as pd
import urllib.request
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
!pip3 install gdown

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
    100% |████████████████████████████████| 61kB 726kB/s ta 0:00:01


In [5]:
!gdown --id 1mTtB8GTWs74Yeqm0KMExGJZh1eDbzUlT --output indian_number_plates.json

Downloading...
From: https://drive.google.com/uc?id=1mTtB8GTWs74Yeqm0KMExGJZh1eDbzUlT
To: /home/pi/Codice/notebooks/indian_number_plates.json
100%|██████████████████████████████████████| 92.5k/92.5k [00:00<00:00, 3.10MB/s]


In [6]:
plates_df = pd.read_json('indian_number_plates.json', lines=True)

In [7]:
import os
os.makedirs("number_plates", exist_ok=True)

In [8]:
dataset = dict()
dataset["image_name"] = list()
dataset["top_x"] = list()
dataset["top_y"] = list()
dataset["bottom_x"] = list()
dataset["bottom_y"] = list()
dataset["class_name"] = list()

In [9]:
#download train and convert the data into the format that Keras Retina understands
counter = 0
for index, row in plates_df.iterrows():
    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert('RGB')
    img.save(f'number_plates/licensed_car_{counter}.jpeg', "JPEG")

    dataset["image_name"].append(
      f'number_plates/licensed_car_{counter}.jpeg'
    )

    data = row["annotation"]

    width = data[0]["imageWidth"]
    height = data[0]["imageHeight"]

    dataset["top_x"].append(int(round(data[0]["points"][0]["x"] * width)))
    dataset["top_y"].append(int(round(data[0]["points"][0]["y"] * height)))
    dataset["bottom_x"].append(int(round(data[0]["points"][1]["x"] * width)))
    dataset["bottom_y"].append(int(round(data[0]["points"][1]["y"] * height)))
    
    dataset["class_name"].append("license_plate")

    counter += 1

In [10]:
print("Downloaded {} car images.".format(counter))


Downloaded 237 car images.


In [11]:
df = pd.DataFrame(dataset)

In [12]:
df

,image_name,top_x,top_y,bottom_x,bottom_y,class_name
0,number_plates/licensed_car_0.jpeg,582,274,700,321,license_plate
1,number_plates/licensed_car_1.jpeg,80,382,288,449,license_plate
2,number_plates/licensed_car_2.jpeg,41,219,115,260,license_plate
3,number_plates/licensed_car_3.jpeg,217,313,359,359,license_plate
4,number_plates/licensed_car_4.jpeg,209,220,866,554,license_plate
5,number_plates/licensed_car_5.jpeg,106,280,261,335,license_plate
6,number_plates/licensed_car_6.jpeg,227,241,444,315,license_plate
7,number_plates/licensed_car_7.jpeg,267,248,396,320,license_plate
8,number_plates/licensed_car_8.jpeg,644,508,893,578,license_plate
9,number_plates/licensed_car_9.jpeg,674,365,958,572,license_plate


In [12]:
RANDOM_SEED = 42
train_df, test_df = train_test_split(df,test_size=0.2,random_state=RANDOM_SEED)

In [13]:
train_df

,image_name,top_x,top_y,bottom_x,bottom_y,class_name
183,number_plates/licensed_car_183.jpeg,169,260,358,298,license_plate
201,number_plates/licensed_car_201.jpeg,251,187,375,234,license_plate
230,number_plates/licensed_car_230.jpeg,140,149,324,220,license_plate
95,number_plates/licensed_car_95.jpeg,302,420,671,534,license_plate
190,number_plates/licensed_car_190.jpeg,115,86,227,117,license_plate
...,...,...,...,...,...,...
106,number_plates/licensed_car_106.jpeg,230,187,327,249,license_plate
14,number_plates/licensed_car_14.jpeg,151,103,360,186,license_plate
92,number_plates/licensed_car_92.jpeg,151,354,303,426,license_plate
179,number_plates/licensed_car_179.jpeg,389,450,600,509,license_plate


In [14]:
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'
#Pandas to write the annotations file, excluding the index and header:
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [15]:
#regular old file writer for the classes:

classes = set(['license_plate'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

In [16]:
os.makedirs("snapshots", exist_ok=True)

In [ ]:
#downloading generic pre-trained-model trained on COCO dataset
!gdown --id 1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R --output snapshots\resnet50_csv_10.h5

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [18]:
#RESUME FROM HERE!
#Getting Resinanet Keras implementation
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [21]:
!pip3 install opencv-python

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
    100% |████████████████████████████████| 10.0MB 22kB/s eta 0:00:01   14% |████▌                           | 1.4MB 3.2MB/s eta 0:00:03


In [25]:
!pip3 install keras-retinanet

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
    100% |████████████████████████████████| 174kB 485kB/s ta 0:00:01
    100% |████████████████████████████████| 4.8MB 35kB/s eta 0:00:01


In [32]:
!sudo apt-get install -y libatlas-base-dev
!sudo apt-get install -y libjasper-dev
!sudo apt-get install -y libqtgui4
!sudo apt-get install -y python3-pyqt5
#!sudo apt-get install -y libgtk2.0-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-8+rpi1).
0 upgraded, 0 newly installed, 0 to remove and 160 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libjasper1
Suggested packages:
  libjasper-runtime
The following NEW packages will be installed:
  libjasper-dev libjasper1
0 upgraded, 2 newly installed, 0 to remove and 160 not upgraded.
Need to get 611 kB of archives.
After this operation, 1,207 kB of additional disk space will be used.
Get:1 http://raspbian.mirror.garr.it/mirrors/raspbian/raspbian buster/main armhf libjasper1 armhf 1.900.1-debian1-2.4+deb8u1 [110 kB]
Get:2 http://raspbian.mirror.garr.it/mirrors/raspbian/raspbian buster/main armhf libjasper-dev armhf 1.900.1-debian1-2.4+deb8u1 [501 kB]
Fetched 611 kB in 1s (474 kB/s)    
Selecting previously unse

Selecting previously unselected package python3-sip.
Preparing to unpack .../python3-sip_4.19.14+dfsg-2_armhf.deb ...
Unpacking python3-sip (4.19.14+dfsg-2) ...
Selecting previously unselected package python3-pyqt5.
Preparing to unpack .../python3-pyqt5_5.11.3+dfsg-1+rpi1+b3_armhf.deb ...
Unpacking python3-pyqt5 (5.11.3+dfsg-1+rpi1+b3) ...
Setting up libqt5designer5:armhf (5.11.3-4) ...
Setting up libqt5help5:armhf (5.11.3-4) ...
Setting up libqt5test5:armhf (5.11.3+dfsg1-1+rpi1) ...
Setting up python3-sip (4.19.14+dfsg-2) ...
Setting up python3-pyqt5 (5.11.3+dfsg-1+rpi1+b3) ...
Processing triggers for libc-bin (2.28-10+rpi1) ...


In [36]:
!sudo apt install -y libqt4-test

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libqt4-test
0 upgraded, 1 newly installed, 0 to remove and 156 not upgraded.
Need to get 97.8 kB of archives.
After this operation, 250 kB of additional disk space will be used.
Get:1 http://ftp.arnes.si/mirrors/raspbian/raspbian buster/main armhf libqt4-test armhf 4:4.8.7+dfsg-18+rpi1 [97.8 kB]33m
Fetched 97.8 kB in 8s (12.9 kB/s)      

78Selecting previously unselected package libqt4-test:armhf.
(Reading database ... 103318 files and directories currently installed.)
Preparing to unpack .../libqt4-test_4%3a4.8.7+dfsg-18+rpi1_armhf.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking libqt4-test:armhf (4:4.8.7+dfsg-18+rpi1) ...
7Progress: [ 40%] [#######################...................................] 8Setti

In [37]:
#training model
!python3.7 ~/.local/lib/python3.7/site-packages/keras_retinanet/bin/train.py \
 --freeze-backbone \
 --random-transform \
 --weights {PRETRAINED_MODEL} \
 --batch-size 8 \
 --steps 500 \
 --epochs 10 \
 csv annotations.csv classes.csv

Using TensorFlow backend.
2020-01-13 14:24:14.666601: E tensorflow/core/platform/hadoop/hadoop_file_system.cc:132] HadoopFileSystem load error: libhdfs.so: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.7/site-packages/keras_retinanet/bin/train.py", line 39, in <module>
    from ..callbacks.eval import Evaluate
  File "/home/pi/.local/lib/python3.7/site-packages/keras_retinanet/bin/../../keras_retinanet/callbacks/eval.py", line 18, in <module>
    from ..utils.eval import evaluate
  File "/home/pi/.local/lib/python3.7/site-packages/keras_retinanet/bin/../../keras_retinanet/utils/eval.py", line 18, in <module>
    from .visualization import draw_detections, draw_annotations
  File "/home/pi/.local/lib/python3.7/site-packages/keras_retinanet/bin/../../keras_retinanet/utils/visualization.py", line 17, in <module>
    import cv2
  File "/home/pi/.local/lib/python3.7/site-packages/keras_retinanet/bin/../../cv2

In [ ]:
#loading the model 

model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

In [ ]:
#convert the classes into a format that will be useful later:
labels_to_names = pd.read_csv(
  CLASSES_FILE,
  header=None
).T.loc[0].to_dict()

In [ ]:
#using model:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels